## Running QISKit Jupyter Notebooks using the IBM Data Science Experience

This article shows you how to set up and run [QISKit](https://www.qiskit.org/) Jupyter notebooks using [IBM Data Science Experience](https://datascience.ibm.com/) (IBM DSX), thanks to Doug McClure ([@dtmcclure](https://github.com/dtmcclure)) and Ninad Sathaye ([@sathayen](https://github.com/sathayen/)). The latest version of this notebook is available [here](https://github.com/QISKit/qiskit-tutorial), where you can find all the other QISKit tutorials.

#### About IBM Data Science Experience

IBM DSX is an interactive, collaborative, cloud-based environment where data scientists can use multiple tools to activate their insights.  

If you are just starting with QISKit and looking for a webhosted Jupyter notebook environment, IBM DSX is an excellent option. It is fairly easy to get your quantum notebook running in the Jupyter notebook environment provided by IBM DSX. Moreover, it provides a platform where you can create your notebooks, invite fellow researchers to collaborate, or simply share your work within the community.

### Prerequisites

- You must sign up for the IBM Data Science Experience ([IBM DSX](https://datascience.ibm.com/)). Tip: It is often convenient to use an IBM ID when signing up. If you do not have one already, [create it first](https://www.ibm.com/account/us-en/signup/register.html).
- You also need an account for the IBM Quantum Experience ([IBM QX](https://quantumexperience.ng.bluemix.net/qx)). You can optionally use the IBM ID here as well. This account is required so that you can connect to the API using an 'API TOKEN'. 

### Steps
1. [Setup QISKit](#setup)
2. [Test QISKit](#test)
3. [Run QISKit](#run) 

#### 1. Setting up QISKit in your DSX account <a id='setup'></a>
1. Download this file ([DSX.ipynb](DSX.ipynb))
2. Log into the [IBM DSX](https://datascience.ibm.com/)
3. Create a new notebook from your downloaded DSX.ipynb
4. Get your IBM QX API token
  - Logon to the [IBM QX](https://quantumexperience.ng.bluemix.net/qx)
  - Click on your username on the top right, and select *My Account*
  - Copy the token displayed under *Advanced*. Make sure it is enclosed in quotes ""
5. Update the value *`"PUT_YOUR_API_TOKEN_HERE"`* in the code cell with the API token you just copied in previous step
  - **NOTE:** If you are sharing this notebook, MAKE SURE TO REMOVE the API token string before sharing! 
6. Run the following code cell to install QISKit on your DSX account and connect to the IBM QX. 

In [ ]:
# This cell does some preparatory work to set QISKit up on the IBM Data Science Experience. 

# --------------------------------------------------------------------------------------------------
# IMPORTANT NOTES: 
# 1) Your QX API token can be found in the *Advanced* section of the *My Account* page of the QX. 
#    Copy-Paste it into the QX_API_TOKEN variable below. BE SURE TO ENCLOSE IN QUOTES ("")
# 2) If you are sharing any notebooks, MAKE SURE TO REMOVE the API token string before sharing! 
# 3) This both creates an IBMQE_API environment variable as well as a Qconfig.py file for connecting
#    to IBM QX.
# --------------------------------------------------------------------------------------------------
import os, sys

os.environ["IBMQE_API"] = "PUT_YOUR_API_TOKEN_HERE"

# DO NOT CHANGE THE FOLLOWING assertions
assert os.environ["IBMQE_API"] != "PUT_YOUR_API_TOKEN_HERE", "QX_API_TOKEN not updated!"

# Install qiskit 
!pip install qiskit

# Create Qconfig.py for connecting to QX
import importlib
import urllib.request

home_dir = os.path.expanduser('~')
qconfig_filepath = os.path.join(home_dir, 'Qconfig.py') 

qconfig_template_path = "https://raw.githubusercontent.com/QISKit/qiskit-tutorial/master/Qconfig.py.template"

# We need visibility to Qconfig.py module. Add home dir in your sys.path just to be sure. 
if home_dir not in sys.path:
    sys.path.append(home_dir)
    
# First check if the Qconfig module has already been loaded (may happen if you are executing this cell more than once)
if 'Qconfig' in sys.modules:
    # The module has been imported already. Reload it to make sure we get the latest API token
    try:
        importlib.reload(Qconfig)
        Qconfig.update_token(os.environ["IBMQE_API"])
    except AttributeError:
        print("Qconfig reload failed. This could be due to missing Qconfig.py file.")
        print("Try restarting the Jupyter notebook kernel!")
        raise
    except AssertionError:
        print("Have you set a valid APItoken?")
        raise
    except:
        print("Qconfig reload or API token update failed.")
        print("Try updating the token and restarting the Jupyter notebook kernel")
        raise 
else:
    # Try importing Qconfig module. If it doesn't exist, then download it from the qiskit-tutorials repo. 
    try:
        import Qconfig
    except ImportError:
        urllib.request.urlretrieve ("{}".format(qconfig_template_path), "{}".format(qconfig_filepath))
        # chmod the file. For Python 3, need to prefix the permission with 0o (zero and character small oh)
        os.chmod(qconfig_filepath , 0o664)
        import Qconfig
    except:
        print("Unexpected error!")
        raise
            

#### 2. Testing your QISKit setup<a id='test'></a>

Run the following *code cell* to test your QISKit setup. It should print `COMPLETED`.

In [ ]:
# Trivial program to test if the QISKit setup was sucessful.
from qiskit import *

qp = QuantumProgram()
qp.set_api(Qconfig.APItoken, Qconfig.config['url'])

# Create a 2-qubit Quantum Register, "qr"
qr = qp.create_quantum_register("qr", 2)
# Create a Classical Register, "cr", with 2 bits
cr = qp.create_classical_register("cr", 2)
# Create the circuit
qc = qp.create_circuit('Bell', [qr], [cr])

# add measure to see the state
qc.measure(qr, cr)

result = qp.execute('Bell', backend='ibmqx_qasm_simulator', shots=1, seed=88)

# Show the results
print(result)

#### 3. Running other QISKit Notebooks<a id='run'></a>

Upload other tutorial notebooks into IBM DSX, copy the following *code cell* after updating the API token and paste it at the beginning of the *other* notebooks.

In [ ]:
# Create IBMQE_API environment variable - remember to remove token if sharing any notebooks
import os, sys
os.environ["IBMQE_API"] = "PUT_YOUR_API_TOKEN_HERE"

assert os.environ["IBMQE_API"] != "PUT_YOUR_API_TOKEN_HERE", "QX_API_TOKEN not updated!"
assert sys.version_info[0:2] >= (3, 5) , "This code requires Python 3.5 or beyond!"

# Add path to and import QConfig.py
home_dir = os.path.expanduser('~')
qconfig_filepath = os.path.join(home_dir, 'Qconfig.py') 

if home_dir not in sys.path:
    sys.path.append(home_dir)

import Qconfig

# NOTE:
# If you ever change the value of environment variable os.environ["IBMQE_API"] AFTER executing this cell, 
# you can call the following code to update its value. 
# Qconfig.update_token(os.environ["IBMQE_API"])    
# Or just set it as : Qconfig.APItoken = os.environ["IBMQE_API"] 
